# Hacettepe University Computer Science Department
## Assignment 3

***

Name and Surname : _Burak Emre Ozer_

Identity Number : _21527266_

Course : _Introduction to Machine Learning Lab. (BBM409)_

Due : _20/12/2018_

Advisors: _Dr. Aykut Erdem, T.A. Necva Bölücü_



## PART 1: Theory Questions

***

### 1. What are differences between logistic regression and linear regression?

The difference between **linear regression** and **logistic regression** is that linear regression is used to predict a _**continuous value**_ while logistic regression is used to predict a _**discrete value**_.  In brief, linear regression is used for regression while logistic regression is used for classification.

### 2. What are differences between logistic regression and naive bayes methods?

Logistic Regression makes a prediction for the probability using a direct functional form where as Naive Bayes figures out how the data was generated given the results. Naive Bayes also assumes that the features are conditionally independent. 

### 3. Which of the following statements are true?

* _False_
* **True**
* _False_
* **True**

### 4. How to decide the number of hidden layers and nodes in a hidden layer?

**Hidden Layer**

* Two or fewer hidden layers will often suffice with simple data sets. However, with complex datasets involving time-series or computer vision, additional layers can be helpful.

**Nodes in a hidden layer**

* The number of hidden neurons should be between the size of the input layer and the size of the output layer.
* The number of hidden neurons should be 2/3 the size of the input layer, plus the size of the output layer.
* The number of hidden neurons should be less than twice the size of the input layer.


## PART 2: Classification of Flowers using Neural Network

***

### Aim

We will implement a simple single layer neural network and multilayer neural network
architecture to classify flowers into 5 classses which daisy, tulip, rose, sunflower, dandelion.

### Data source

<img src="images/dataset.png">


This dataset contains 4242 images of flowers. The data collection is based on the data flickr, google images, yandex images. 

The pictures are divided into five classes: daisy, tulip, rose, sunflower, dandelion. For each class there are about 800 photos. 



## 1.  Single Layer Neural Network

The architecture of this single-layer neural network has 768 inputs and 5 outputs. 

I used the **softmax** function which is uses for the multiclass logistic regression. Softmax is kind of Multi Class Sigmoid basically getting a probability of each class. I also used **cross-entropy** to calculate the loss. 

Let's overview of the core point of the code:

```python
    def train(self):
        X = self.X
        for i in np.arange(0, self.epoch):
            epoch_loss_values = []
            # train the data using mini-batch
            for current_batch in np.arange(0, X.shape[0], self.batch_size):
                bX = X[current_batch:current_batch + self.batch_size]
                batch_labels = self.y[current_batch:current_batch + self.batch_size]

                outputs = self.feedforward(bX)
                probs = self.softmax(outputs) # getting a probability of each class

                loss = self.cross_entropy_loss(bX, batch_labels, probs) # calc. the loss using cross-entropy
                epoch_loss_values.append(loss)

                self.backpropagation(bX, batch_labels, probs) # minimize the loss func. using gradient descent
            self.loss.append(np.mean(epoch_loss_values))

        model = [self.weights, self.b]
        np.save("slnn_model", model) # save the model current directory
```

## 1.1 Software Usage

* _firstly, run train.py to train the model_:
```python
python3 train.py -data_path path/to/data -e epoch number -a learning rate -b batch size```

* _then run test.py to calculate the results_:
```python
python3 test.py -data_path path/to/data -model_path path/to/model```

## 1.2 Visualization of parameters

<img src="images/paramsr.png">

If we pay attention to the sunflower and the tulip, the model looks well trained. But the accuracy rates overlap with this situation.

## 1.3 Analyzing the experiments and their effects

#### 1.3.1 Activation function

<img src="images/act_func.png" width="400px">

Different activation functions, in fact, do have different properties. I use softmax for the output layer and cross entropy as a cost function. Also I used sigmoid or tanh with squared error as a cost function. As seen from the plot, the most successful activation function was reLU for this problem. 

#### 1.3.2 Epoch number effect

As the number of epochs increases, more number of times the weight are changed in the neural network and the curve goes from underfitting to optimal to overfitting curve.

|epoch| learning rate |batch size| accuracy |
|:---:|:---:|:---:|:---:|
|50|0.2|128|0.32|
|100|0.2|128|0.35|
|250|0.2|128|0.32|
|500|0.2|128|0.31|

| &nbsp;|&nbsp; |
:-------------------------:|:-------------------------:
<img src="images/e50alp002.png" width="250px">  |  <img src="images/e100alp002.png" width="250px"> |
<img src="images/e250alp002.png" width="250px">|  <img src="images/e500alp002.png" width="250px">

***

#### 1.3.3 Learning rate effect

<img src="images/alpha.png" align="right" width="200px">

The learning rate affects how quickly our model can converge to a local minima. If it is too big, may not converge; if it is too small, it occurs very slow convergence. Thus getting it right from the get go would mean lesser time for us to train the model.

|epoch| learning rate |batch size| accuracy |
|:---:|:---:|:---:|:---:|
|100|0.005|128|0.32|
|100|0.01|128|0.33|
|100|0.015|128|0.34|
|100|0.02|128|0.35|


| &nbsp;|&nbsp; |
:-------------------------:|:-------------------------
<img src="images/a0005.png" width="250px">  |  <img src="images/a001.png" width="250px"> |  
<img src="images/a0015.png" width="250px">|  <img src="images/a002.png" width="250px">

***

#### 1.3.4 Batch size effect

Total number of training examples present in a single batch. With a small batch size, the gradients are only a very rough approximation of the true gradients.

|epoch| learning rate |batch size| accuracy |
|:---:|:---:|:---:|:---:|
|250|0.015|16|0.31|
|250|0.015|32|0.32|
|250|0.015|64|0.31|
|250|0.015|128|0.34|


| &nbsp;|&nbsp; |
:-------------------------:|:-------------------------:
<img src="images/b16.png" width="250px">  |  <img src="images/b32.png" width="250px"> |
<img src="images/b64.png" width="250px">|  <img src="images/b128.png" width="250px">

## 1.  Multi Layer Neural Network

The architecture of this multi-layer neural network has 1 hidden layer and output layer. 
Hidden layer node size: 100


Let's overview of the core point of the code:

```python
    class MultiLayer:
        def __init__(self, x, y, epoch, alpha, batch_size):
            self.W = np.random.rand(self.X.shape[1], 50) / 500
            self.W2 = np.random.rand(50, 5) / 500
            self.b = np.zeros((1, 50))
            self.b2 = np.zeros((1, 5))
            
        def feedforward(self, bX):
            # using reLu activation func.
            hidden_layer = np.maximum(0, np.dot(bX, self.W) + self.b)  
            scores = np.dot(hidden_layer, self.W2) + self.b2
            return hidden_layer, scores
            
        def backpropagation(self, bX, bY, probs, hidden_layer):
            #calculate gradient descent using by derivative of softmax
            d_outputs = probs
            d_outputs[range(bX.shape[0]), bY] -= 1
            d_outputs /= bX.shape[0]

            dW2 = np.dot(hidden_layer.T, d_outputs)
            db2 = np.sum(d_outputs, axis=0, keepdims=True)
            #backprop. to hidden layer
            d_hidden = np.dot(d_outputs, self.W2.T)
            # then use derivative of the reLu activation func.
            d_hidden[hidden_layer <= 0] = 0
            dW = np.dot(bX.T, d_hidden)
            db = np.sum(dhidden, axis=0, keepdims=True)
            # updating the parameters
            self.W += -self.alpha * dW
            self.b += -self.alpha * db
            self.W2 += -self.alpha * dW2
            self.b2 += -self.alpha * db2

```

## 1.1 Software Usage

Please the uncomment the multilayer part and comment single layer part in train.py if you want to use multi layer neural network.

* _firstly, run train.py to train the model_:
```python
python3 train.py -data_path path/to/data -e epoch number -a learning rate -b batch size```

* _then run test.py to calculate the results_:
```python
python3 test.py -data_path path/to/data -model_path path/to/model```


## 1.2 Analyzing the experiments and their effects

Although I tried different parameters in the multi layer section, I couldn't get the results I expected. So I could not make the necessary improvements. I couldn't find out what I did wrong. :(

<img src="images/multi_results.png" width="400px">

#### 1.2.1 Learning rate effect


|epoch| learning rate |batch size| accuracy |
|:---:|:---:|:---:|:---:|
|300|0.005|128|0.33|
|300|0.01|128|0.36|
|300|0.015|128|0.35|
|300|0.02|128|0.34|


| &nbsp;|&nbsp; |&nbsp; |&nbsp; |
:-------------------------:|:-------------------------|:-------------------------|:-------------------------
<img src="images/ma0005.png" width="250px">  |  <img src="images/ma001.png" width="250px"> |  <img src="images/ma0015.png" width="250px">|  <img src="images/ma002.png" width="250px">


#### 1.3.4 Batch size effect


| &nbsp;|&nbsp; |&nbsp; |&nbsp; |
:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:
<img src="images/mb16.png" width="250px">  |  <img src="images/mb32.png" width="250px"> | <img src="images/mb64.png" width="250px"> |  <img src="images/mb128.png" width="250px">


## Resources

https://medium.com/@michaeldelsole/a-single-layer-artificial-neural-network-in-20-lines-of-python-ae34b47e5fef

https://enlight.nyc/projects/neural-network/

https://medium.com/analytics-vidhya/neural-networks-for-digits-recognition-e11d9dff00d5

http://cs231n.github.io/neural-networks-case-study/#grad

https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/src/network.py

https://medium.freecodecamp.org/building-a-3-layer-neural-network-from-scratch-99239c4af5d3